## Весь препроцессинг вместе

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import json as js
import csv

In [2]:
df = pd.read_csv('data/resume.csv', index_col=0)

In [3]:
df.shape

(2369, 19)

In [4]:
df.head(10)

,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"['Немецкий язык, Базовый']",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...
1,39,['B — легковые авто'],NaN,Техническая поддержка,Россия,Москва,"[{'kind': 'Высшее образование', 'form': 'Дневн...",полная занятость,"[{'period': 'октябрь 2017 – работает сейчас', ...",18 лет,cостоит в браке,m,NaN,NaN,"Администратор, управляющий, IT специалист",https://public.superjob.ru/images/resume_fotos...,50 000₽,[{'professional': '• 1С: Предприятие • 1С: Роз...,https://www.superjob.ru/resume/administrator-2...
2,47,NaN,NaN,Передача данных и доступ в интернет,NaN,Новосибирск,"[{'kind': 'Высшее образование', 'form': 'Вечер...",полная занятость,"[{'period': 'февраль 2008 – работает сейчас', ...",12 лет и 10 месяцев,"cостоит в браке, есть дети",m,"['Английский язык, Технический']",Берёзовая роща,Администратор / Инженер технической поддержки IT,NaN,30 000₽,[{'professional': 'Английский язык уровень. Pr...,https://www.superjob.ru/resume/administrator-2...
3,23,NaN,NaN,Техническая поддержка,NaN,Самара,"[{'kind': 'Высшее образование', 'form': 'Дневн...",сменный график работы,"[{'period': 'июль 2015 – работает сейчас', 'le...",4 года и 3 месяца,NaN,f,NaN,NaN,"Администратор, IT-специалист, бизнес-аналитик,...",https://public.superjob.ru/images/resume_fotos...,25 000₽,"[{'professional': 'Веб-дизайн, photoshop, MS O...",https://www.superjob.ru/resume/administrator-3...
4,52,NaN,не готов к командировкам,АХО,Россия,Ангарск,"[{'kind': 'Высшее образование', 'form': 'Дневн...",NaN,"[{'period': 'июнь 1993 – март 2015', 'length':...",21 год и 10 месяцев,"не состоит в браке, есть дети",f,NaN,NaN,"Администратор, IT-специалист, диспетчер",NaN,20 000₽,NaN,https://www.superjob.ru/resume/administrator-3...
5,22,NaN,NaN,Системное администрирование,Россия,Москва,"[{'kind': 'Неполное высшее образование', 'form...",NaN,"[{'period': 'сентябрь 2017 – декабрь 2017', 'l...",1 год и 2 месяца,NaN,m,NaN,готов к переезду,"Администратор, менеджер, IT сфера",https://public.superjob.ru/images/resume_fotos...,45 000₽,"[{'professional': 'Отличное знание ПК (Exel, 1...",https://www.superjob.ru/resume/administrator-4...
6,23,NaN,NaN,Техническая поддержка,Россия,Миллерово,"[{'kind': 'Среднее специальное образование', '...",полная занятость,"[{'period': 'август 2015 – ноябрь 2017', 'leng...",2 года и 4 месяца,"не состоит в браке, детей нет",m,"['Английский язык, Базовый']",готов к переезду,"Администратор, IT-специалист",https://public.superjob.ru/images/resume_fotos...,20 000₽,[{'professional': 'Поверхностное знание програ...,https://www.superjob.ru/resume/administrator-4...
7,24,"['A — мотоциклы', 'B — легковые авто']",NaN,Сетевые технологии,Россия,Санкт-Петербург,"[{'kind': 'Бакалавр', 'form': 'Дистанционная',...",полная занятость,"[{'period': 'декабрь 2017 – работает сейчас', ...",4 года и 7 месяцев,NaN,m,"['Английский язык, Базовый', 'Эстонский язык, ...",NaN,"Администратор компьютерных сетей, техник IT",NaN,45 000₽,"[{'professional': 'Диагностика, ремонт неиспра...",https://www.superjob.ru/resume/administrator-k...
8,30,NaN,не готов к командировкам,Складское хозяйство,Россия,Железнодорожный,"[{'kind': 'Высшее образование', 'form': 'Очно-...",полная занятость,"[{'period': 'август 2017 – сентябрь 2018', 'le...",6 лет и 2 месяца,cостоит в браке,m,"['Английский язык, Базовый']",NaN,"Администратор магазина, кладовщик, старший кла...",NaN,50 000₽,NaN,https://www.superjob.ru/resume/administrator-m...
9,22,NaN,NaN,NaN,Россия,Рязань,"[{'kind': 'Высшее образование', 'form': 'Очно-...",полная занятость,"[{'period': 'июнь 2014 – август 2018', 'length...",4 года и 3

In [5]:
df.columns

Index(['age', 'auto', 'business_trips', 'categories', 'citizenship', 'city',
       'education', 'employment', 'experience', 'experience_length', 'family',
       'gender', 'languages', 'metro_or_moving', 'name', 'photo_url', 'salary',
       'skills', 'url'],
      dtype='object')

## Обработка колонок:

1. `age` - Возраст
2. `auto` - Наличие прав
3. `business_trips` - Готовность к командировкам $\longrightarrow$ **удалили**
4. `categories` - Категории сайта $\longrightarrow$ `categories_text`
5. `citizenship` - Гражданство $\longrightarrow$ `rus_citizenship`
6. `city` - Город - дало 3 новых признака $\longrightarrow$ `city_msk`, `city_spb`, `city_other`
7. `education` - Образование $\longrightarrow$ `education_degree`
8. `employment` - Желаемая занятость $\longrightarrow$ `full_employment`
9. `experience` - Опыт работы $\longrightarrow$ `experience_text`
10. `experience_length` - Продолжительность опыта работы
11. `family` - Семейное положение $\longrightarrow$ **удалили**
12. `gender` - Пол
13. `languages` - Владение языками $\longrightarrow$ `english_level`
14. `metro_or_moving` - Станция метро или готовность к переезду
15. `name` - Название вакансии $\longrightarrow$ `name_text`
16. `photo_url` - Ссылка на фотографию  $\longrightarrow$ **удалили**
17. `salary`  - Зарплата
18. `skills` - Ключевые навыки $\longrightarrow$ `skills_text`
19. `url` - Ссылка на резюме  $\longrightarrow$ **удалили**

## 1. `age`

In [6]:
 df.age.value_counts(dropna=False)

34    99
35    99
33    98
32    96
31    95
29    93
30    92
41    85
28    85
36    83
27    82
39    75
24    74
37    74
38    73
23    72
26    68
25    67
40    63
22    62
21    61
45    59
43    58
42    56
46    51
44    51
48    45
47    34
49    33
20    31
54    30
52    30
51    27
56    27
55    26
53    26
50    22
58    13
19     9
57     9
59     7
62     5
64     5
60     3
61     3
63     3
18     3
72     2
67     1
66     1
68     1
65     1
70     1
Name: age, dtype: int64

## 2. `auto`

In [7]:
df.auto.value_counts(dropna=False)

['B — легковые авто']                                                                                 1171
NaN                                                                                                    808
['B — легковые авто', 'C — грузовые авто']                                                             220
['A — мотоциклы', 'B — легковые авто']                                                                  74
['A — мотоциклы', 'B — легковые авто', 'C — грузовые авто']                                             52
['C — грузовые авто']                                                                                   11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы']                                              11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы', 'E — авто с прицепом']                        7
['B — легковые авто', 'D — автобусы']                                                                    5
['A — мотоциклы', 'B — легковые авто'

In [8]:
df.auto.loc[~df.auto.isnull()] = 1
df.auto.loc[df.auto.isnull()] = 0

/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## 3. `business_trips`

In [9]:
df.business_trips.value_counts(dropna=False)

NaN                         1373
не готов к командировкам     968
на дому                       28
Name: business_trips, dtype: int64

Много NaN, выкидываем.

In [10]:
df = df.drop('business_trips', axis=1)

## 4. `categories`

In [11]:
df.categories.value_counts(dropna=False)

Системное администрирование                      552
NaN                                              467
Техническая поддержка                            358
IT, Интернет, связь, телеком                     238
Инжиниринг                                       184
Администрирование баз данных                      80
Административная работа, секретариат, АХО         64
Сетевые технологии                                49
Разработка, программирование                      41
Передача данных и доступ в интернет               38
Управление проектами                              34
Аналитика                                         31
Интернет, создание и поддержка сайтов             23
Другое                                            18
Электронный документооборот                       15
Внедрение и сопровождение ПО                      12
Делопроизводство, ввод данных, систематизация     11
Компьютеры, оргтехника, ПО                         9
Системная интеграция                          

In [12]:
df['categories_text'] = df.categories
df.categories_text = df.categories_text.fillna('')
df = df.drop('categories', axis=1)

## 5. `citizenship`

In [13]:
df.citizenship.value_counts(dropna=False)

Россия               1615
NaN                   727
Украина                 8
Казахстан               7
Азербайджан             3
Кыргызстан              3
Беларусь                2
Литва                   1
Дальнее зарубежье       1
Молдова                 1
Таджикистан             1
Name: citizenship, dtype: int64

Будем полагать, что те, кто не указал гражданство имеют гражданство России.

In [14]:
df.citizenship = df.citizenship.fillna('Россия')
citizenship = pd.get_dummies(df.citizenship)
df['rus_citizenship'] = citizenship['Россия']
df = df.drop('citizenship', axis=1)

In [15]:
df.rus_citizenship.value_counts(dropna=False)

1    2342
0      27
Name: rus_citizenship, dtype: int64

## 6. `city`

In [16]:
df.city.value_counts(dropna=False)

Москва                                 720
Санкт-Петербург                        293
Хабаровск                               64
Уфа                                     55
Иркутск                                 51
Ростов-на-Дону                          48
Краснодар                               33
Самара                                  32
Саратов                                 27
Казань                                  27
Рязань                                  26
Нижний Новгород                         24
Ульяновск                               23
Пенза                                   23
Волгоград                               22
Воронеж                                 20
Оренбург                                19
Сыктывкар                               17
Тверь                                   17
Екатеринбург                            16
Челябинск                               16
Владимир                                16
Смоленск                                16
Астрахань  

Оставим 3 признака: Москва, Санкт-Петербург, другие регионы

In [17]:
ctgr = pd.get_dummies(df.city)
ctgr
city_msk = ctgr['Москва']
city_spb = ctgr['Санкт-Петербург']
ctgr = ctgr.drop(['Москва', 'Санкт-Петербург'], axis=1)
city_other = np.sum(ctgr.values, axis=1)
df['city_msk'] = city_msk
df['city_spb'] = city_spb
df['city_other'] = city_other
df = df.drop(['city'], axis=1)
df.shape

(2369, 20)

## 7. `education`

In [18]:
df.education.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                68
[{'kind': 'Среднее образование'}]                                                                                                                                 

In [19]:
def i_am_crab(x):
    if type(x)==float:
        return 0
    else:
        max_val = 0
        for y in eval(x):
            if (y['kind'])=='Среднее образование' and max_val < 1:
                max_val = 1
           
            if (y['kind'])=='Среднее специальное образование' and max_val < 2:
                max_val = 2
            if (y['kind'])=='Неполное высшее образование' and max_val < 3:
                max_val = 3
               
            if (y['kind'])=='Бакалавр' and max_val < 4:
                max_val = 4
               
            if (y['kind'])=='Магистр' and max_val < 5:
                max_val = 5
            if (y['kind'])=='Кандидат наук' and max_val < 6:
                max_val = 6
        return max_val

In [20]:
df['education_degree'] = df.education.apply(i_am_crab)
df = df.drop('education', axis=1)

## 8. `employment`

In [21]:
df.employment.value_counts(dropna=False)

полная занятость           1180
NaN                        1107
неполный рабочий день        33
сменный график работы        25
частичная занятость          13
работа вахтовым методом      11
Name: employment, dtype: int64

In [22]:
ctgr = pd.get_dummies(df.employment)
fn = lambda row: 1 if(row['полная занятость'] == 1 or row['сменный график работы'] == 1)else 0
full_empl = ctgr.apply(fn, axis=1)
df['full_employment'] = full_empl
df = df.drop('employment', axis=1)

## 9. `experience`

In [23]:
import ast

def exp_to_text(exp):
    if exp != exp: # check for nan
        return ''
    arr = ast.literal_eval(exp)
    result = ''
    for exp_dict in arr:
        if 'name' in exp_dict:
            result += exp_dict['name'] + ' '
        if 'duties' in exp_dict:
            result += exp_dict['duties'] + ' '
    return result

In [24]:
df['experience_text'] = df.experience.apply(exp_to_text)
df = df.drop('experience', axis=1)

## 10. `experience_length`

In [25]:
df.experience_length = df.experience_length.fillna('0 лет и 0 месяцев')
df.experience_length.value_counts(dropna=False)

0 лет и 0 месяцев       126
7 лет и 3 месяца         19
14 лет и 4 месяца        18
12 лет и 8 месяцев       15
11 лет и 11 месяцев      15
11 лет и 4 месяца        15
10 лет и 6 месяцев       15
5 лет и 10 месяцев       15
9 лет и 7 месяцев        15
10 лет и 2 месяца        14
3 года и 6 месяцев       14
13 лет и 11 месяцев      14
7 лет и 8 месяцев        14
6 лет                    14
7 лет и 2 месяца         14
16 лет и 2 месяца        13
1 год и 4 месяца         13
2 года и 5 месяцев       13
6 лет и 6 месяцев        13
10 лет и 3 месяца        13
9 лет и 6 месяцев        13
1 год и 3 месяца         12
8 лет и 6 месяцев        12
16 лет и 5 месяцев       12
12 лет и 5 месяцев       12
9 лет и 11 месяцев       12
2 года и 10 месяцев      12
6 лет и 11 месяцев       12
12 лет и 11 месяцев      12
7 лет и 9 месяцев        12
                       ... 
37 лет и 2 месяца         1
25 лет и 4 месяца         1
41 год и 11 месяцев       1
25 лет и 6 месяцев        1
29 лет и 8 месяцев  

In [26]:
df.shape

(2369, 20)

In [27]:
df.index = np.arange(df.shape[0])

In [28]:
def period_to_month(string):
    s = string.split()
    if len(s) == 2:
        if (s[1] == 'год') or (s[1] == 'года') or (s[1] == 'лет'):
            return int(s[0]) * 12
        return int(s[0])
    return int(s[0]) * 12 + int(s[3])

In [29]:
pers = df.experience_length.copy()

In [30]:
pers is df.experience_length

False

In [31]:
for i in pers.index:
    pers.values[i] = period_to_month(pers.values[i])

In [32]:
df.experience_length = pers
df.experience_length

0         7
1       216
2       154
3        51
4       262
5        14
6        28
7        55
8        74
9        51
10       59
11      224
12        3
13       78
14      172
15      361
16      411
17      111
18      250
19        0
20      183
21      284
22      216
23      280
24      381
25      186
26      126
27      128
28       42
29      268
       ... 
2339    121
2340    283
2341    283
2342    207
2343    244
2344    167
2345    121
2346    179
2347    165
2348    389
2349    334
2350    245
2351     97
2352    194
2353    211
2354    154
2355    268
2356     75
2357    298
2358    156
2359    194
2360    339
2361    156
2362    146
2363     66
2364    180
2365    347
2366    229
2367     34
2368     72
Name: experience_length, Length: 2369, dtype: object

## 11. `family`

In [33]:
df.family.value_counts(dropna=False)

cостоит в браке, есть дети       804
NaN                              687
не состоит в браке, детей нет    528
cостоит в браке, детей нет       168
не состоит в браке, есть дети    111
cостоит в браке                   37
не состоит в браке                29
есть дети                          3
детей нет                          2
Name: family, dtype: int64

In [34]:
df = df.drop('family', axis=1)

## 12. `gender`

In [35]:
gender = pd.get_dummies(df.gender)
df['gender'] = gender.f

## 13. `languages`

In [36]:
df.languages.value_counts(dropna=False)

['Английский язык, Технический']                                                                                                                               832
NaN                                                                                                                                                            547
['Английский язык, Базовый']                                                                                                                                   463
['Английский язык, Разговорный']                                                                                                                               164
['Английский язык, Технический', 'Немецкий язык, Базовый']                                                                                                      47
['Английский язык, Базовый', 'Немецкий язык, Базовый']                                                                                                          42
['Английский язык, Cво

In [37]:
df.languages=df.languages.fillna("[]")

In [38]:
def i_am_cruel(x):
    #print (eval(x))
    for z in eval(x):
        y = z.split(', ')
        if y[0] == 'Английский язык':
            if y[1] == 'Базовый':
                return 1
            if y[1] == 'Технический':
                return 2
            if y[1] == 'Разговорный':
                return 3
            if y[1] == 'Cвободное владение':
                return 4
            print (y[1])
    return 0
for i in df.languages.values:
    i_am_cruel(i)
df['english_level']=df.languages.apply(i_am_cruel)

In [39]:
df.drop('languages', axis=1, inplace=True)

## 14. `metro_or_moving`

In [40]:
df.metro_or_moving.value_counts(dropna=False)

NaN                                               1357
готов к переезду                                   540
Пражская                                            12
Улица Дыбенко                                        9
Комендантский проспект                               8
Новогиреево                                          6
Коломенская                                          6
Первомайская                                         6
Выхино                                               6
Ботанический садВДНХ                                 6
Медведково                                           6
Проспект Ветеранов                                   6
Ладожская                                            6
Новокосино                                           5
Проспект Большевиков                                 5
Щелковская                                           5
Академическая                                        5
Планерная                                            5
Люблино   

Половина Nan-ов, удаляем.

In [41]:
df.drop('metro_or_moving', axis=1, inplace=True)

## 15. `name`

In [42]:
df.name.value_counts(dropna=False)

Системный администратор, IT-специалист                                                               416
IT-специалист                                                                                        207
IT специалист                                                                                         76
ИТ-специалист                                                                                         54
Руководитель IT-отдела                                                                                37
Инженер IT                                                                                            29
Руководитель отдела ИТ                                                                                27
Руководитель IT отдела                                                                                24
It-специалист                                                                                         22
Ведущий специалист ИТ                                  

In [43]:
df['name_text'] = df.name
df = df.drop('name', axis=1)

## 16. `photo_url`

In [44]:
df = df.drop('photo_url', axis=1)

## 17. `salary`

In [45]:
df.salary=df.salary.apply(lambda x: int(x[:-1].replace(" ", '')))

In [46]:
df.salary.value_counts

<bound method IndexOpsMixin.value_counts of 0        30000
1        50000
2        30000
3        25000
4        20000
5        45000
6        20000
7        45000
8        50000
9        30000
10       20000
11       30000
12       35000
13       60000
14       30000
15       75000
16       75000
17       20000
18      150000
19       25000
20      150000
21       70000
22      280000
23      240000
24       90000
25       80000
26       20000
27       50000
28       50000
29      100000
         ...  
2339    175000
2340     80000
2341     80000
2342    150000
2343    200000
2344     50000
2345    120000
2346     70000
2347    130000
2348     50000
2349    120000
2350     55000
2351     50000
2352    100000
2353    150000
2354    120000
2355     50000
2356     25000
2357     60000
2358    100000
2359     50000
2360     50000
2361     60000
2362    110000
2363     30000
2364    150000
2365    150000
2366     40000
2367     45000
2368     80000
Name: salary, Length: 2369, dtype: int64>

## 18. `skills`

In [47]:
df.skills.value_counts(dropna=False).shape

(1965,)

Часть - дубликаты. Но полных дубликатов не было. Видимо, люди хотят на разные должности по одинаковым резюме. Можно оставить как есть (логике задачи это не противоречит)

In [48]:
df.groupby(by='skills')

In [49]:
df[df.skills.isna()].shape

(330, 17)

In [50]:
df = df.dropna(subset=['skills'])

In [51]:
df.shape

(2039, 17)

In [52]:
def skill_to_text(skill):
    if skill != skill: # check for nan
        return ''
    arr = ast.literal_eval(skill)
    result = ''
    exp_dict = arr[0]
    if 'professional' in exp_dict:
        result += exp_dict['professional'] + ' '
    return result

In [53]:
df['skills_text'] = df.skills.apply(skill_to_text)
df = df.drop('skills', axis=1)

/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## 19. `url`

In [54]:
df = df.drop('url', axis = 1)

## Заполненность резюме

Добавим новый признак - длина строк, отвечающих за требования на прошлых местах работы и скиллы.

Название резюме обычно всегда фиксированной длины.

Категории заполняются не соискателем.

In [55]:
def resume_fullfillment(resume):
    result = 0
    if resume['experience_text'] != '':
        result += len(resume['experience_text'])
    if resume.skills_text != '':
        result += len(resume.skills_text)
    return result

In [56]:
df['fullfillment'] = df.apply(resume_fullfillment, axis=1)

## Ну и объединим все текстовые признаки в один

In [57]:
columns = list(df.columns.values)
text_cols = filter(lambda c: c.endswith('_text'), columns)
text_cols = list(text_cols)
text_cols

['categories_text', 'experience_text', 'name_text', 'skills_text']

In [58]:
def join_text(resume):
    result = ''
    for text_col in text_cols:
        if resume[text_col] != '' or resume[text_col] == resume[text_col]:
            result += resume[text_col] + ' '
    return result

In [59]:
df['text'] = df.apply(join_text, axis=1)
df = df.drop(text_cols, axis=1)

## Результат:

In [60]:
print(df.shape)
df.head(10)

(2039, 14)


,age,auto,experience_length,gender,salary,rus_citizenship,city_msk,city_spb,city_other,education_degree,full_employment,english_level,fullfillment,text
1,39,1,216,0,50000,1,1,0,0,0,1,0,1676,Техническая поддержка Администратор Организаци...
2,47,0,154,0,30000,1,0,0,1,0,1,2,961,Передача данных и доступ в интернет Старший ин...
3,23,0,51,1,25000,1,0,0,1,0,1,0,299,Техническая поддержка Продавец-кассир Экскурсо...
5,22,0,14,0,45000,1,1,0,0,3,0,0,761,Системное администрирование Специалист офиса п...
6,23,0,28,0,20000,1,0,0,1,2,1,1,513,Техническая поддержка Оператор ЭВМ В данной ор...
7,24,1,55,0,45000,1,0,1,0,4,1,1,984,"Сетевые технологии Системный администратор, IT..."
9,22,0,51,0,30000,1,0,0,1,0,1,2,939,Методист по информационным технологиям Устано...
10,31,1,59,0,20000,1,0,0,1,2,0,1,5431,Web-верстка Администратор сообщества ВКонтакте...
11,48,0,224,0,30000,1,0,0,1,0,1,2,1588,Сетевые технологии Системный администратор Адм...
12,23,1,3,0,35000,1,1,0,0,4,1,1,295,Техническая поддержка Электроник 1ой категории...


In [61]:
df.columns

Index(['age', 'auto', 'experience_length', 'gender', 'salary',
       'rus_citizenship', 'city_msk', 'city_spb', 'city_other',
       'education_degree', 'full_employment', 'english_level', 'fullfillment',
       'text'],
      dtype='object')

In [62]:
with open('data/preprocessed_resume.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv)

## Имеем фичи:

1. `age` - Возраст - **нужно нормализовать**
2. `auto` - Наличие прав
3. `experience_length` - Продолжительность опыта работы - **нужно нормализовать**
4. `gender` - Пол
5. `salary`  - Зарплата
6. `rus_citizenship` - Наличие российского гражданства
7. `city_msk` - Живет в Москве
8. `city_spb` - Живет в Санкт-Петербурге
9. `city_other` - Живет не в Москве и не в Санкт-Петербурге
10. `education_degree` - Уровень образования
11. `full_employment` - Готовность работать на полную ставку
12. `english_level` - Уровень владения английским
13. `fullfillment` - Заполненность пунктов, содержащих ключевые навыки - **нужно нормализовать**
14. `text` - Текст, содержащий ключевые навыки - **нужно вычленить ключевые навыки**

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [64]:
import matplotlib.pyplot as plt

%matplotlib inline

#plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)

In [93]:
bag = vectorizer.fit_transform(df.text)
bag = bag.toarray()
bag.shape

(2039, 35442)

In [94]:
x = df.drop(['salary', 'text'], axis=1).values.copy()
x.shape

(2039, 12)

In [96]:
X = np.concatenate((x, bag), 1)
X.shape

(2039, 35454)

In [69]:
from sklearn.cluster import AgglomerativeClustering
clust = AgglomerativeClustering()
clust

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=2,
            pooling_func=<function mean at 0x7f1caad719d8>)

In [147]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(min_samples_leaf=10, max_features=3000, n_estimators=1000, max_depth=15)
regressor

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=3000, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [148]:
regressor.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=3000, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [149]:
regressor.score(X, y)

0.6435394815678035

In [150]:
y_pr = regressor.predict(X).reshape(-1, 1)
y.mean()

64689.271701814614

In [151]:
np.concatenate((y_pr, y.reshape(-1, 1)), axis=1)[-10:]

array([[ 49818.08217377,  50000.        ],
       [ 82487.69706581,  50000.        ],
       [ 67327.2055106 ,  60000.        ],
       [ 78795.10535364, 110000.        ],
       [ 49039.11609193,  30000.        ],
       [138019.92198133, 150000.        ],
       [123648.04283151, 150000.        ],
       [ 47181.31726801,  40000.        ],
       [ 49180.44148381,  45000.        ],
       [ 53214.75237556,  80000.        ]])

In [157]:
regressor.feature_importances_.reshape(-1, 1)

array([[0.0099345 ],
       [0.00125321],
       [0.03689939],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [162]:
X_notxt = df.drop(['salary', 'text'], axis=1).values.copy()
regressor2 = RandomForestRegressor(min_samples_leaf=10, max_features=None, n_estimators=1000, max_depth=30)
regressor2.fit(X_notxt, y)
regressor2.score(X_notxt, y)

0.5385315445223124